In [5]:
import cv2
import os
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from keras.preprocessing.image import load_img, img_to_array, save_img
import imageio
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
import shutil
from tensorflow.keras.backend import clear_session

clear_session()


In [6]:
# Load data path
data_path = "/home/meth/Documents/ETH/DCM/labelled_data" 

In [7]:
all_patients = sorted([f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))])
random_seed = 42
tf.random.set_seed(random_seed)
train_val_patients, test_patients = train_test_split(all_patients, test_size=0.1, random_state=random_seed)

train_patients, val_patients = train_test_split(train_val_patients, test_size=50/450, random_state=random_seed)

print(f"Training Patients: {len(train_patients)}")  
print(f"Validation Patients: {len(val_patients)}")  
print(f"Testing Patients: {len(test_patients)}")  
print(f"Total Patients: {len(all_patients)}")


Training Patients: 400
Validation Patients: 50
Testing Patients: 50
Total Patients: 500


In [8]:
# Create a folder to save test patients
test_patients_folder = "/home/meth/Documents/ETH/DCM/ed_test_data" # Change this path as needed
if not os.path.exists(test_patients_folder):
    os.makedirs(test_patients_folder)

# Copy test patients folders to the new test folder
for patient in test_patients:
    patient_path = os.path.join("/home/meth/Documents/ETH/DCM/labelled_data" , patient)  # Adjust this path to your actual patient data location
    if os.path.exists(patient_path):
        dest_path = os.path.join(test_patients_folder, patient)
        shutil.copytree(patient_path, dest_path)
    else:
        print(f"Patient folder {patient} not found.")

print(f"All test patients have been copied to {test_patients_folder}")

All test patients have been copied to /home/meth/Documents/ETH/DCM/ed_test_data


In [9]:
patient_ids = []  
for patient_id in os.listdir(data_path):
    patient_path = os.path.join(data_path, patient_id)
    if os.path.isdir(patient_path):
        patient_ids.append(patient_id)  
print(patient_ids[:3]) 

['patient0222', 'patient0320', 'patient0046']


In [10]:
#function to threshold the mask
def threshold_mask(mask):
    """Convert a grayscale mask to a binary mask with values 0 and 1."""
    _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    binary_mask = binary_mask / 255  # Convert 255 to 1, so values become either 0 or 1
    return binary_mask

In [11]:
def load_patient_data(patient_ids, root_path, target_size=(256, 256)):
    images, masks = [], []
    
    for patient_id in patient_ids:
        ed_image_dir = os.path.join(root_path, patient_id, "ED", "images")
        ed_mask_dir = os.path.join(root_path, patient_id, "ED", "masks")
        
        if os.path.exists(ed_image_dir) and os.path.exists(ed_mask_dir):
            #print(f"Found images in {ed_image_dir}")
            #print(f"Found masks in {ed_mask_dir}")
            
            image_files = [f for f in os.listdir(ed_image_dir) if f.endswith('.png') and not f.startswith('.')]
            mask_files = [f for f in os.listdir(ed_mask_dir) if f.endswith('.png') and not f.startswith('.')]
            
            #print(f"Image files in {ed_image_dir}: {image_files}")
            #print(f"Mask files in {ed_mask_dir}: {mask_files}")
            
            for image_file in image_files:
                mask_file = image_file.replace('.png', '_gt.png')
                if mask_file in mask_files:
                    img_path = os.path.join(ed_image_dir, image_file)
                    mask_path = os.path.join(ed_mask_dir, mask_file)
                    
                    try:
                        image = Image.open(img_path).convert('RGB')
                        image_resized = image.resize(target_size, Image.BICUBIC)
                        binary_images = np.array(image_resized)
                        normalized_image = binary_images/255
                        images.append(np.array(normalized_image))
                        
                        mask = Image.open(mask_path).convert('L')
                        mask_resized = mask.resize(target_size, Image.NEAREST)
                        mask_binary = np.array(mask_resized)

                        mask_binary = (mask_binary > 128).astype(np.uint8)
                        masks.append(mask_binary)
                    except Exception as e:
                        # print(f"Error loading image or mask {img_path}/{mask_path}: {e}")
                        continue
    
    print(f"Total images loaded: {len(images)}")
    print(f"Total masks loaded: {len(masks)}")
    
    images = np.array(images)
    masks = np.array(masks)
    
    images = images.reshape(-1, 256, 256, 3)
    masks = masks.reshape(-1, 256, 256, 1)
    
    return images, masks


In [12]:
images, masks = load_patient_data(patient_ids, data_path)
print(f"Number of images loaded: {len(images)}")
print(f"Number of masks loaded: {len(masks)}")



Total images loaded: 500
Total masks loaded: 500
Number of images loaded: 500
Number of masks loaded: 500


In [13]:
def unet_model(input_shape=(256, 256, 1), learning_rate=1e-4):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D(pool_size=(2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1], axis=3)
    c9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    
    return model
# Create model
model = unet_model()
model.summary()

W0000 00:00:1749634342.276499    7081 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │      9,248 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │     36,928 │ conv2d_2[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │    147,584 │ conv2d_4[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32,    │    295,168 │ max_pooling2d_2[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 32, 32,    │    590,080 │ conv2d_6[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 16, 16,    │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 16, 16,    │  1,180,160 │ max_pooling2d_3[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 16, 16,    │  2,359,808 │ conv2d_8[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 32, 32,    │    524,544 │ conv2d_9[0][0]    │
│ (Conv2DTranspose)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 32,    │          0 │ conv2d_transpose

 Total params: 7,759,521 (29.60 MB)

 Trainable params: 7,759,521 (29.60 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def iou_metric(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)


In [15]:
folds = []
num_folds = 9 
fold_size = len(train_val_patients) // num_folds  

for fold in range(num_folds):
    fold_train_patients = np.concatenate([train_val_patients[:fold * fold_size], train_val_patients[(fold + 1) * fold_size:]])
    fold_val_patients = train_val_patients[fold * fold_size : (fold + 1) * fold_size]

    folds.append((fold_train_patients, fold_val_patients))

dice_scores = []
ious = []

for fold in range(num_folds):
    print(f"\nFold {fold + 1}:")

    fold_train_patients, fold_val_patients = folds[fold]

    X_train, y_train = load_patient_data(fold_train_patients, data_path)
    X_val, y_val = load_patient_data(fold_val_patients, data_path)
    X_test, y_test = load_patient_data(test_patients, data_path) 

    y_train = np.expand_dims(y_train, axis=-1)
    y_val = np.expand_dims(y_val, axis=-1)
    y_test = np.expand_dims(y_test, axis=-1) 

    print(f"Fold Training Patients: {len(fold_train_patients)}")
    print(f"Fold Validation Patients: {len(fold_val_patients)}")

    model = unet_model(input_shape=X_train.shape[1:])

    # Dice loss function
    def dice_loss(y_true, y_pred):
        numerator = 2 * K.sum(y_true * y_pred)
        denominator = K.sum(y_true + y_pred)
        return 1 - numerator / (denominator + K.epsilon())
    
    # def custom_loss(y_true, y_pred):
    #     cross_entropy_loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)(y_true, y_pred)
    #     weight_decay_loss = tf.reduce_sum([tf.nn.l2_loss(w) for w in model.trainable_weights])
    #     return cross_entropy_loss + 1e-4 * weight_decay_loss 


Fold 1:
Total images loaded: 400
Total masks loaded: 400
Total images loaded: 50
Total masks loaded: 50
Total images loaded: 50
Total masks loaded: 50
Fold Training Patients: 400
Fold Validation Patients: 50

Fold 2:
Total images loaded: 400
Total masks loaded: 400
Total images loaded: 50
Total masks loaded: 50
Total images loaded: 50
Total masks loaded: 50
Fold Training Patients: 400
Fold Validation Patients: 50

Fold 3:
Total images loaded: 400
Total masks loaded: 400
Total images loaded: 50
Total masks loaded: 50
Total images loaded: 50
Total masks loaded: 50
Fold Training Patients: 400
Fold Validation Patients: 50

Fold 4:
Total images loaded: 400
Total masks loaded: 400
Total images loaded: 50
Total masks loaded: 50
Total images loaded: 50
Total masks loaded: 50
Fold Training Patients: 400
Fold Validation Patients: 50

Fold 5:
Total images loaded: 400
Total masks loaded: 400
Total images loaded: 50
Total masks loaded: 50
Total images loaded: 50
Total masks loaded: 50
Fold Trainin

In [16]:
# model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss=custom_loss, metrics=['accuracy', dice_coefficient, iou_metric])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss=dice_loss,
                  metrics=['accuracy',dice_coefficient, iou_metric])

checkpoint = ModelCheckpoint(f"unet_fold_{fold + 1}.keras", save_best_only=True, monitor='val_loss', mode='min')

history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=80,
        batch_size=8,
        callbacks=[checkpoint]
    )

Epoch 1/80


/home/meth/miniconda3/envs/milly/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_288']. Received: the structure of inputs=*
  warnings.warn(


50/50 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.5574 - dice_coefficient: 0.1835 - iou_metric: 0.1012 - loss: 0.8165 - val_accuracy: 0.5637 - val_dice_coefficient: 0.2230 - val_iou_metric: 0.1256 - val_loss: 0.7724
Epoch 2/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 120s 2s/step - accuracy: 0.6611 - dice_coefficient: 0.3598 - iou_metric: 0.2287 - loss: 0.6402 - val_accuracy: 0.9206 - val_dice_coefficient: 0.6633 - val_iou_metric: 0.4967 - val_loss: 0.3405
Epoch 3/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 120s 2s/step - accuracy: 0.9208 - dice_coefficient: 0.6736 - iou_metric: 0.5091 - loss: 0.3264 - val_accuracy: 0.9230 - val_dice_coefficient: 0.7012 - val_iou_metric: 0.5402 - val_loss: 0.3011
Epoch 4/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 120s 2s/step - accuracy: 0.9373 - dice_coefficient: 0.7287 - iou_metric: 0.5743 - loss: 0.2713 - val_accuracy: 0.9568 - val_dice_coefficient: 0.8010 - val_iou_metric: 0.6682 - val_loss: 0.1984
Epoch 5/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - accuracy: 0.9593 - dice_coefficien

In [17]:
val_loss, val_accuracy, val_dice_score, val_iou = model.evaluate(X_val, y_val)
print(f"Fold {fold + 1} - Validation Loss: {val_loss}, Accuracy: {val_accuracy}, Dice Score: {val_dice_score}, IoU: {val_iou}")

dice_scores.append(val_dice_score)
ious.append(val_iou)

test_loss, test_accuracy, test_dice_score, test_iou = model.evaluate(X_test, y_test)
print(f"Fold {fold + 1} - Test Loss: {test_loss}, Test Accuracy: {test_accuracy}, Test Dice Score: {test_dice_score}, Test IoU: {test_iou}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9848 - dice_coefficient: 0.9335 - iou_metric: 0.8754 - loss: 0.0667
Fold 9 - Validation Loss: 0.06638483703136444, Accuracy: 0.9851678609848022, Dice Score: 0.9340391159057617, IoU: 0.8762452602386475
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9863 - dice_coefficient: 0.9389 - iou_metric: 0.8848 - loss: 0.0607
Fold 9 - Test Loss: 0.06125675514340401, Test Accuracy: 0.986096203327179, Test Dice Score: 0.9380679130554199, Test IoU: 0.8833693861961365


In [18]:
mean_dice = np.mean(dice_scores)
std_dice = np.std(dice_scores)
mean_iou = np.mean(ious)
std_iou = np.std(ious)

print(f"Mean Dice Score: {mean_dice}, Standard Deviation: {std_dice}")
print(f"Mean IoU: {mean_iou}, Standard Deviation: {std_iou}")


Mean Dice Score: 0.9340391159057617, Standard Deviation: 0.0
Mean IoU: 0.8762452602386475, Standard Deviation: 0.0


In [19]:
model.save(f"ed_unet_model_b8_ep80_{mean_dice:.4f}.keras")